In [7]:
!pip install feedparser
!pip install bs4
!pip install urllib.parse

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement urllib.parse (from versions: none)
ERROR: No matching distribution found for urllib.parse


First using Feedparser to get links

In [5]:
import feedparser
feed = feedparser.parse('https://bg.raindrop.io/rss/public/47223284')

print([(e.title, e.link) for e in feed.entries])
print(f"Found {len(feed.entries)} entries")

entries = []
#entries is now our URL list from RSS feed
for e in feed.entries:
    title = getattr(e, "title", "")
    link = getattr(e, "link", "")
    entries.append({"title":title, "link":link})

[('Intro DS Syllabi at 2-year Colleges', 'https://docs.google.com/spreadsheets/d/1wihCoaD5Ei-11kxzowDjp_v0gXX5LNb_DdXwsBswknw/edit?gid=0#gid=0'), ('Introduction to Data Science | University of Stavanger', 'https://www.uis.no/en/course/DAT540_1'), ('Open Access Data Science Resource by Data Science Discovery at the University of Illinois', 'https://discovery.cs.illinois.edu/'), ('Columbia | Foundations of Data Science: Syllabus', 'https://www.columbia.edu/~cs2035/courses/orca2500.S18/syllabus.html'), ('Delta College MTH 225 - Introduction to Data Science - Modern Campus Catalog™', 'https://catalog.delta.edu/preview_course_nopop.php?catoid=15&coid=31564'), ('Data Science (DS) 210 | Johnson County Community College Catalog', 'https://catalog.jccc.edu/coursedescriptions/ds/#DS_210'), ('Undergrad Intro DS Syllabi', 'https://docs.google.com/spreadsheets/d/1HTnAukzyXh0wM411MMeiNjR-O1sXTmIwX8GqzQgPxIY/edit?gid=0#gid=0'), ('C S 8A: INTRODUCTION TO DATA SCIENCE Foothill College', 'https://catalo

In [10]:
from bs4 import BeautifulSoup
import requests
import json


def extract_text_from_url(url) -> dict:
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    soup = BeautifulSoup(response.text, 'html.parser')
    # Remove script and style elements
    for tag in soup(['script', 'style']):
        tag.decompose()

    # Get text
    text = soup.get_text(separator='\n')
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    return {
        "url": url,
        "text": "\n".join(lines)
    }

In [11]:
urls = [entry['link'] for entry in entries]

with open('rss_extracted_texts.jsonl', 'w', encoding='utf-8') as f:
    for url in urls:
        try:
            extracted = extract_text_from_url(url)
            f.write(json.dumps(extracted, ensure_ascii=False) + '\n')
        except Exception as e:
            print(f"Failed to extract {url}: {e}")

Failed to extract https://www.columbia.edu/~cs2035/courses/orca2500.S18/syllabus.html: 403 Client Error: Forbidden for url: https://www.columbia.edu/~cs2035/courses/orca2500.S18/syllabus.html
Failed to extract https://fan.princeton.edu/fan/classes/525.html: 403 Client Error: Forbidden for url: https://fan.princeton.edu/fan/classes/525.html


In [12]:
docs = []
with open('rss_extracted_texts.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        docs.append(json.loads(line)["text"])
print(f"Extracted {len(docs)} documents.")

Extracted 30 documents.


In [13]:
import re
def clean_text(t):
    t = t.lower()
    t = re.sub(r'\s+', ' ', t)  # Replace multiple whitespace with single space
    t = re.sub(r'[^a-z0-9\s]', '', t)  # Remove non-alphanumeric characters
    return t.strip()
cleaned_docs = [clean_text(doc) for doc in docs]
print(cleaned_docs[0][:500])  # Print first 500 characters of the first

intro ds syllabi at 2year colleges  google sheets javascript isnt enabled in your browser so this file cant be opened enable and reload this browser version is no longer supported please upgrade to a supported browser intro ds syllabi at 2year colleges tab external share sign in file edit view insert format data tools extensions help accessibility debug unsaved changes to drive accessibility view only loading a b c d e f g h i j k l m n o p q r s t u v w x y z 1 course name department college pr


TF-IDF to see dominant terms per course

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf_vectorizer = TfidfVectorizer(
    max_features = 5000,
    stop_words='english',
    ngram_range=(1,2)
)

X = tfidf_vectorizer.fit_transform(cleaned_docs)

#top terms globally
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_means = X.mean(axis=0).A1

top = pd.DataFrame({
    'term': feature_names,
    'tfidf': tfidf_means
}).sort_values(by='tfidf', ascending=False).head(30)

print(top)

                   term     tfidf
1308               data  0.193209
3931            science  0.079908
1235             course  0.076207
1344       data science  0.070035
4406           students  0.053151
1027              class  0.045188
641         assignments  0.041192
3655             python  0.039753
2493                lab  0.037275
4817                use  0.034547
1067               code  0.033417
2584           learning  0.031096
2377       introduction  0.030351
2085           homework  0.028954
3626            project  0.027242
2112              hours  0.027191
930             catalog  0.025981
491            analysis  0.025846
1733              final  0.024902
628          assignment  0.024622
2615            lecture  0.024401
4939               work  0.024232
2168                ids  0.024187
4916               week  0.024007
3615        programming  0.023897
4832              using  0.023799
2380  introduction data  0.023188
1146           computer  0.023009
3590        pr